In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [3]:
df = pd.read_csv('SoftwareDefect.csv')
print(df.shape)
df.head()


(10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True


In [4]:
print(df.describe())
print(df.info())
#print(df['target'].value_counts())

                loc          v(g)         ev(g)         iv(g)             n  \
count  10885.000000  10885.000000  10885.000000  10885.000000  10885.000000   
mean      42.016178      6.348590      3.401047      4.001599    114.389738   
std       76.593332     13.019695      6.771869      9.116889    249.502091   
min        1.000000      1.000000      1.000000      1.000000      0.000000   
25%       11.000000      2.000000      1.000000      1.000000     14.000000   
50%       23.000000      3.000000      1.000000      2.000000     49.000000   
75%       46.000000      7.000000      3.000000      4.000000    119.000000   
max     3442.000000    470.000000    165.000000    402.000000   8441.000000   

                  v             l             d             i             e  \
count  10885.000000  10885.000000  10885.000000  10885.000000  1.088500e+04   
mean     673.758017      0.135335     14.177237     29.439544  3.683637e+04   
std     1938.856196      0.160538     18.709900    

In [ ]:
sns.pairplot(df)
sns.heatmap(df.corr(), annot=True)

In [ ]:
df = df.dropna()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop('target', axis=1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, df['target'], test_size=0.2, random_state=42)